## 1.BERT 和 Decoder-only 模型（如 GPT、LLaMA）的分词算法有什么区别？


BERT 采用 WordPiece 分词算法，非首段子词会添加 ## 前缀以标记拼接关系，且会在输入中插入 [CLS]（句首分类标记）和 [SEP]（句子分隔标记）这两个人工定义的特殊功能符号；GPT-1/2/3 采用 BPE 分词算法，LLaMA 采用 Byte-level BPE 分词算法，二者子词均无额外前缀、呈独立形式，且主要使用 <|endoftext|>（文本结束标记）作为核心特殊符号。



#### **2. WordPiece和BPE的关键区别对比**
| **对比维度**| **WordPiece (BERT)**| **BPE (GPT/LLaMA)**|
|--------------------|-----------------------------------------------|----------------------------------------------|
| **子词标记**| 子词带 `##` 前缀（如 `"playing" → ["play", "##ing"]`） | 无前缀，子词独立（如 `"playing" → ["play", "ing"]`） |
| **合并策略**| 基于互信息分数（合并后更像合理词）| 基于频率（合并高频共现对）|
| **特殊标记**| `[CLS]`, `[SEP]`, `[MASK]`（服务于 MLM 和句子对任务） | `<|endoftext|>`（控制生成边界）|
| **空格处理**| 忽略或标准化| 显式编码（如 `Ġ` 或 `▁`）|


#### **3. 合并策略公式与示例**
**WordPiece 是谁组合后更像一个"合理词"就合并谁， 互信息计算公式为**：
$$
\text{score}(A, B) = \frac{\text{freq}(A,B)}{\text{freq}(A) \cdot \text{freq}(B)}
$$
- **例子**：
- 若 `freq(un)=50`, `freq(happy)=20`, `freq(h)=500`：
- `"un"+"happy"` 分数：`100/(50×20) = 0.1`
- `"un"+"h"` 分数：`200/(50×500) = 0.008`
- **合并结果**：优先合并 `"unhappy"`（分数更高）。

**BPE 是谁出现多就合并谁，依据频率合并**：
- **例子**：
- `"un"+"h"` 出现 200 次，`"un"+"happy"` 出现 100 次 → 合并 `"unh"`。


#### **4. 特殊标记的用途**
- **BERT 的特殊标记**：
    - `[CLS]`：分类任务中汇总序列信息。
    - `[SEP]`：分隔句子对（如问答任务）。
    - `[MASK]`：预训练时随机遮盖 Token。
    - **影响**：分词器必须支持这些标记，否则无法正常训练/推理。



- **Decoder-only 的特殊标记**：
    - `<|endoftext|>`：标记文本边界，控制生成终止。
    - **影响**：生成时必须正确处理该标记，否则输出会无限延续。

---

### 5.代码对比区别

#### bert的分词结果

In [10]:
from transformers import BertTokenizer

def wordpiece_tokenize(text):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    inputs = tokenizer(text, return_tensors="pt")

    # 打印输入的token ID
    print("input_ids:", inputs["input_ids"])

    # 将token ID转换为对应的文本标记，直观查看特殊符号
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    print("tokens:", tokens)


text = "Hello world!"
wordpiece_tokenize(text)

text = "unhappiness"
wordpiece_tokenize(text)


input_ids: tensor([[ 101, 7592, 2088,  999,  102]])
tokens: ['[CLS]', 'hello', 'world', '!', '[SEP]']
input_ids: tensor([[ 101, 4895, 3270, 9397, 9961,  102]])
tokens: ['[CLS]', 'un', '##ha', '##pp', '##iness', '[SEP]']


#### gpt2的分词结果

In [14]:
from transformers import GPT2Tokenizer

def gpt2_tokenize(text):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    
    inputs = tokenizer(text, return_tensors="pt")
    
    # 打印输入的token ID
    print("input_ids:", inputs["input_ids"])
    
    # 将token ID转换为对应的文本标记
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    print("tokens:", tokens)


text = "Hello world!"
gpt2_tokenize(text)

text = "unhappiness"
gpt2_tokenize(text)


input_ids: tensor([[15496,   995,     0]])
tokens: ['Hello', 'Ġworld', '!']
input_ids: tensor([[  403,    71, 42661]])
tokens: ['un', 'h', 'appiness']


#### llama的分词结果

In [15]:
from transformers import AutoTokenizer

def llama_tokenize(text):
    # 使用小型LLaMA模型的分词器
    tokenizer = AutoTokenizer.from_pretrained("openlm-research/open_llama_3b")
    
    inputs = tokenizer(text, return_tensors="pt")
    
    # 打印输入的token ID
    print("input_ids:", inputs["input_ids"])
    
    # 将token ID转换为对应的文本标记
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    print("tokens:", tokens)


text = "Hello world!"
llama_tokenize(text)

text = "unhappiness"
llama_tokenize(text)


input_ids: tensor([[    1, 16644,   924, 31905]])
tokens: ['<s>', '▁Hello', '▁world', '!']
input_ids: tensor([[    1, 16640,  1323,   919]])
tokens: ['<s>', '▁unh', 'app', 'iness']
